# EDA 및 파생변수 추가

In [ ]:
# EDA 및 파생변수 추가
# 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
data = pd.read_csv("https://firebasestorage.googleapis.com/v0/b/ls-storage-e452a.appspot.com/o/casting.csv?alt=media&token=54fffc33-03ad-4431-8e20-34ede32c043f",
                      encoding='cp949',
                      index_col=0)
data.nunique()
# data = pd.read_csv('/content/drive/MyDrive/lsda02/data/casting.csv', encoding='cp949', index_col=0)

data.head()
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install seaborn
!pip install matplotlib
!pip install xgboost
!pip install lightgbm
!pip install eli5
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy
import xgboost
import lightgbm
import warnings
from eli5.sklearn import PermutationImportance
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from lightgbm import LGBMClassifier, plot_importance
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedShuffleSplit
warnings.filterwarnings('ignore')
data.hist(figsize=(25,20))
plt.show()
pass_count = len(data[data['passorfail']== 0])
fail_count = len(data[data['passorfail']== 1])
print('양품 개수 : ', pass_count)
print('불량 개수 : ', fail_count)
print('불량률 :', fail_count/(fail_count+pass_count)*100)
use_col = data.dtypes[data.dtypes!='object'].index
data = data[use_col]
data
data_copy = data.copy()
data_copy
data_copy.columns
data_copy.nunique()
data_copy.shape
data_copy.isnull().sum()
data_copy.describe().T
data_copy.info()
data_copy.drop('molten_volume', axis=1, inplace=True)
data_copy.dropna(axis=0, inplace=True)
data_copy.reset_index(drop=True, inplace=True)
data_copy
for col in data_copy.dtypes[data_copy.dtypes!='object'].index:
  UCL = np.percentile(data_copy[col], 99.9)
  LCL = np.percentile(data_copy[col], 0.1)
  data_copy = data_copy[(data_copy[col] <= UCL) & (data_copy[col] >= LCL)]

data_copy.reset_index(drop=True, inplace=True)
data_copy
t_test = []
for i, col in enumerate(data_copy.columns):
  t=scipy.stats.ttest_ind(data_copy[data_copy['passorfail']==1][col], data_copy[data_copy['passorfail']==0][col],
equal_var=False)
  t_test.append([col, t[0], t[1]])
df_ttest = pd.DataFrame(t_test, columns=['col', 'tvalue', 'pvalue'])
use_t_list = []
for i in range(len(df_ttest)):
  if df_ttest['pvalue'][i] < 0.1:
    use_t_list.append(df_ttest['col'][i])
df_ttest
data_copy = data_copy[use_t_list]
data_copy
data_copy.drop(['count', 'EMS_operation_time', 'mold_code'], axis=1, inplace=True)
data_copy
plt.figure(figsize=(25, 25))
sns.heatmap(data_copy.corr(), annot = True, fmt = '.1f', square = True, cmap = 'Blues')
plt.show()
X = data_copy.drop('passorfail', axis=1).values
y = data_copy['passorfail'].values
sss = StratifiedShuffleSplit(n_splits=5, random_state=42, test_size=0.2)
mms = MinMaxScaler()
dt_clf = DecisionTreeClassifier(random_state=42,max_depth=4)
rf_clf = RandomForestClassifier(random_state=42)
xgb_clf = XGBClassifier(random_state=42, eval_metric='error', use_label_encoder=False)
lgbm_clf = LGBMClassifier(random_state=42)
ada_clf = AdaBoostClassifier(random_state=42)
dt_clf = DecisionTreeClassifier(random_state=42)
dt_fold_f1 = []
n_iter = 0
print('모델 :', dt_clf)
idx_list = []

for train_idx, test_idx in sss.split(X, y):
  X_train, X_test = X[train_idx], X[test_idx]
  y_train, y_test = y[train_idx], y[test_idx]
  for train_idx, valid_idx in sss.split(X_train, y_train):
    X_valid = X_train[valid_idx]
    y_valid = y_train[valid_idx]
  X_train_scaled = mms.fit_transform(X_train)
  X_test_scaled = mms.transform(X_test)
  X_valid_scaled = mms.transform(X_valid)
  dt_clf.fit(X_train_scaled, y_train)
  valid_pred = dt_clf.predict(X_valid_scaled)
  test_pred = dt_clf.predict(X_test_scaled)
  valid_f1 = f1_score(y_valid, valid_pred)
  test_f1 = f1_score(y_test, test_pred)
  n_iter += 1
  dt_fold_f1.append([n_iter, test_f1])
  idx_list.append([train_idx, test_idx])
  print(f'StratifiedShuffleSplit {n_iter} 검증 데이터 f1 정확도 : {valid_f1}')
  print(f'StratifiedShuffleSplit {n_iter} 테스트 데이터 f1 정확도 : {test_f1}')
  print('='*50)
  dt_fold_f1.sort(key=lambda x : x[1], reverse=True)
  print(f'교차검증 중 가장 높은 정확도 : Split {dt_fold_f1[0][0]} {dt_fold_f1[0][1]}')
dt_clf.fit(X[idx_list[dt_fold_f1[0][0]-1][0]], y[idx_list[dt_fold_f1[0][0]-1][0]])
pred = dt_clf.predict(X[idx_list[dt_fold_f1[0][0]-1][1]])
cm = confusion_matrix(y[idx_list[dt_fold_f1[0][0]-1][1]], pred)
sns.heatmap(cm, annot=True, fmt='.0f')
plt.show()
data_copy.columns
import graphviz
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier

export_graphviz(dt_clf, out_file="tree.dot",
                class_names=['0','1'],
                feature_names = data_copy.columns[:-1],
                impurity=True,
                filled=True)
# 위에서 생성된 tree.dot 파일을 Graphviz 읽어서 Jupyter Notebook상에서 시각화
with open("tree.dot") as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)
# from sklearn.tree import export_graphviz
# import graphviz

# # Decision Tree 모델을 dot 파일로 저장
# export_graphviz(dt_clf, out_file="tree.dot",
#                 class_names=['0', '1'],
#                 feature_names=data_copy.columns[:-1],
#                 impurity=True,
#                 filled=True)

# # tree.dot 파일을 Graphviz로 읽어서 Jupyter Notebook 상에서 시각화
# with open("tree.dot") as f:
#     dot_graph = f.read()

# # dot_graph 문자열에서 그래프 크기를 설정
# dot_graph = dot_graph.replace('size="5,5!"', 'size="8,8!"')  # 원하는 크기로 조정

# # 그래프 출력
# src = graphviz.Source(dot_graph, format='png')
# src.render('tree', format='png', view=True)

# # Jupyter Notebook 상에서 출력
# src

rf_clf = RandomForestClassifier(random_state=42)
rf_fold_f1 = []
n_iter = 0
print('모델 :', rf_clf)
idx_list = []
for train_idx, test_idx in sss.split(X, y):
  X_train, X_test = X[train_idx], X[test_idx]
  y_train, y_test = y[train_idx], y[test_idx]
  for train_idx, valid_idx in sss.split(X_train, y_train):
    X_valid = X_train[valid_idx]
    y_valid = y_train[valid_idx]
  X_train_scaled = mms.fit_transform(X_train)
  X_test_scaled = mms.transform(X_test)
  X_valid_scaled = mms.transform(X_valid)
  rf_clf.fit(X_train_scaled, y_train)
  valid_pred = rf_clf.predict(X_valid_scaled)
  test_pred = rf_clf.predict(X_test_scaled)
  valid_f1 = f1_score(y_valid, valid_pred)
  test_f1 = f1_score(y_test, test_pred)
  n_iter += 1
  rf_fold_f1.append([n_iter, test_f1])
  idx_list.append([train_idx, test_idx])
  print(f'StratifiedShuffleSplit {n_iter} 검증 데이터 f1 정확도 : {valid_f1}')
  print(f'StratifiedShuffleSplit {n_iter} 테스트 데이터 f1 정확도 : {test_f1}')
  print('='*50)
  dt_fold_f1.sort(key=lambda x : x[1], reverse=True)
  print(f'교차검증 중 가장 높은 정확도 : Split {rf_fold_f1[0][0]} {rf_fold_f1[0][1]}')
rf_clf.fit(X[idx_list[rf_fold_f1[0][0]-1][0]], y[idx_list[rf_fold_f1[0][0]-1][0]])
pred = rf_clf.predict(X[idx_list[rf_fold_f1[0][0]-1][1]])
cm = confusion_matrix(y[idx_list[rf_fold_f1[0][0]-1][1]], pred)
sns.heatmap(cm, annot=True, fmt='.0f')
plt.show()
# xgb_clf = RandomForestClassifier(random_state=42)
xgb_fold_f1 = []
n_iter = 0
print('모델 :', xgb_clf)
idx_list = []
for train_idx, test_idx in sss.split(X, y):
  X_train, X_test = X[train_idx], X[test_idx]
  y_train, y_test = y[train_idx], y[test_idx]
  for train_idx, valid_idx in sss.split(X_train, y_train):
    X_valid = X_train[valid_idx]
    y_valid = y_train[valid_idx]
  X_train_scaled = mms.fit_transform(X_train)
  X_test_scaled = mms.transform(X_test)
  X_valid_scaled = mms.transform(X_valid)
  xgb_clf.fit(X_train_scaled, y_train)
  valid_pred = xgb_clf.predict(X_valid_scaled)
  test_pred = xgb_clf.predict(X_test_scaled)
  valid_f1 = f1_score(y_valid, valid_pred)
  test_f1 = f1_score(y_test, test_pred)
  n_iter += 1
  xgb_fold_f1.append([n_iter, test_f1])
  idx_list.append([train_idx, test_idx])
  print(f'StratifiedShuffleSplit {n_iter} 검증 데이터 f1 정확도 : {valid_f1}')
  print(f'StratifiedShuffleSplit {n_iter} 테스트 데이터 f1 정확도 : {test_f1}')
  print('='*50)
  xgb_fold_f1.sort(key=lambda x : x[1], reverse=True)
  print(f'교차검증 중 가장 높은 정확도 : Split {xgb_fold_f1[0][0]} {xgb_fold_f1[0][1]}')
xgb_clf.fit(X[idx_list[xgb_fold_f1[0][0]-1][0]], y[idx_list[xgb_fold_f1[0][0]-1][0]])
pred = xgb_clf.predict(X[idx_list[xgb_fold_f1[0][0]-1][1]])
cm = confusion_matrix(y[idx_list[xgb_fold_f1[0][0]-1][1]], pred)
sns.heatmap(cm, annot=True, fmt='.0f')
plt.show()
lgbm_clf = LGBMClassifier(random_state=42)
lgbm_fold_f1 = []
n_iter = 0
print('모델 :', lgbm_clf)
idx_list = []
for train_idx, test_idx in sss.split(X, y):
  X_train, X_test = X[train_idx], X[test_idx]
  y_train, y_test = y[train_idx], y[test_idx]
  for train_idx, valid_idx in sss.split(X_train, y_train):
    X_valid = X_train[valid_idx]
    y_valid = y_train[valid_idx]
  X_train_scaled = mms.fit_transform(X_train)
  X_test_scaled = mms.transform(X_test)
  X_valid_scaled = mms.transform(X_valid)
  lgbm_clf.fit(X_train_scaled, y_train)
  valid_pred = lgbm_clf.predict(X_valid_scaled)
  test_pred = lgbm_clf.predict(X_test_scaled)
  valid_f1 = f1_score(y_valid, valid_pred)
  test_f1 = f1_score(y_test, test_pred)
  n_iter += 1
  lgbm_fold_f1.append([n_iter, test_f1])
  idx_list.append([train_idx, test_idx])
  print(f'StratifiedShuffleSplit {n_iter} 검증 데이터 f1 정확도 : {valid_f1}')
  print(f'StratifiedShuffleSplit {n_iter} 테스트 데이터 f1 정확도 : {test_f1}')
  print('='*50)
  lgbm_fold_f1.sort(key=lambda x : x[1], reverse=True)
  print(f'교차검증 중 가장 높은 정확도 : Split {lgbm_fold_f1[0][0]} {lgbm_fold_f1[0][1]}')
lgbm_clf.fit(X[idx_list[lgbm_fold_f1[0][0]-1][0]], y[idx_list[lgbm_fold_f1[0][0]-1][0]])
pred = lgbm_clf.predict(X[idx_list[lgbm_fold_f1[0][0]-1][1]])
cm = confusion_matrix(y[idx_list[lgbm_fold_f1[0][0]-1][1]], pred)
sns.heatmap(cm, annot=True, fmt='.0f')
plt.show()
# ada_clf = LGBMClassifier(random_state=42)
ada_fold_f1 = []
n_iter = 0
print('모델 :', lgbm_clf)
idx_list = []
for train_idx, test_idx in sss.split(X, y):
  X_train, X_test = X[train_idx], X[test_idx]
  y_train, y_test = y[train_idx], y[test_idx]
  for train_idx, valid_idx in sss.split(X_train, y_train):
    X_valid = X_train[valid_idx]
    y_valid = y_train[valid_idx]
  X_train_scaled = mms.fit_transform(X_train)
  X_test_scaled = mms.transform(X_test)
  X_valid_scaled = mms.transform(X_valid)
  ada_clf.fit(X_train_scaled, y_train)
  valid_pred = ada_clf.predict(X_valid_scaled)
  test_pred = ada_clf.predict(X_test_scaled)
  valid_f1 = f1_score(y_valid, valid_pred)
  test_f1 = f1_score(y_test, test_pred)
  n_iter += 1
  ada_fold_f1.append([n_iter, test_f1])
  idx_list.append([train_idx, test_idx])
  print(f'StratifiedShuffleSplit {n_iter} 검증 데이터 f1 정확도 : {valid_f1}')
  print(f'StratifiedShuffleSplit {n_iter} 테스트 데이터 f1 정확도 : {test_f1}')
  print('='*50)
  lgbm_fold_f1.sort(key=lambda x : x[1], reverse=True)
  print(f'교차검증 중 가장 높은 정확도 : Split {ada_fold_f1[0][0]} {ada_fold_f1[0][1]}')
ada_clf.fit(X[idx_list[ada_fold_f1[0][0]-1][0]], y[idx_list[ada_fold_f1[0][0]-1][0]])
pred = ada_clf.predict(X[idx_list[ada_fold_f1[0][0]-1][1]])
cm = confusion_matrix(y[idx_list[ada_fold_f1[0][0]-1][1]], pred)
sns.heatmap(cm, annot=True, fmt='.0f')
plt.show()
pi = PermutationImportance(lgbm_clf, scoring='f1', n_iter=30)
pi.fit(X[idx_list[lgbm_fold_f1[0][0]-1][0]], y[idx_list[lgbm_fold_f1[0][0]-1][0]])
permu_df = pd.DataFrame()
permu_df["feature"] = data_copy.columns[:-1]
permu_df["importance_weight"] = pi.feature_importances_
permu_df.sort_values(by='importance_weight', ascending=False, inplace=True)
permu_df.reset_index(drop=True, inplace=True)
plt.figure(figsize=(10, 8))
sns.barplot(x='importance_weight', y='feature', data=permu_df)
plt.title('Permutaion Importance', fontsize=18)
plt.show()
lgbm = LGBMClassifier(random_state=42)
tmp = pd.DataFrame(X[idx_list[lgbm_fold_f1[0][0]-1][0]], columns=data_copy.columns[:-1])
lgbm.fit(tmp, y[idx_list[lgbm_fold_f1[0][0]-1][0]])
lightgbm.plot_tree(lgbm,dpi=100,orientation='vertical',figsize=(20,15), show_info=['internal_value', 'internal_count'])
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv("https://firebasestorage.googleapis.com/v0/b/ls-storage-e452a.appspot.com/o/casting.csv?alt=media&token=54fffc33-03ad-4431-8e20-34ede32c043f",
                   encoding='cp949',
                   index_col=0)

# 연속형 변수 확인
continuous_vars = data.select_dtypes(include=['float64', 'int64']).columns

# passorfail 별로 히스토그램 그리기
for var in continuous_vars:
    plt.figure(figsize=(10, 6))
    for label in data['passorfail'].unique():
        subset = data[data['passorfail'] == label]
        plt.hist(subset[var], bins=30, alpha=0.5, label=f'{label}')

    plt.title(f'Histogram of {var} by passorfail')
    plt.xlabel(var)
    plt.ylabel('Frequency')
    plt.legend(title='passorfail')
    plt.show()

import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv("https://firebasestorage.googleapis.com/v0/b/ls-storage-e452a.appspot.com/o/casting.csv?alt=media&token=54fffc33-03ad-4431-8e20-34ede32c043f",
                   encoding='cp949',
                   index_col=0)

# 연속형 변수 확인
continuous_vars = data.select_dtypes(include=['float64', 'int64']).columns

# 서브플롯 설정
fig, axes = plt.subplots(nrows=7, ncols=3, figsize=(15, 30))
axes = axes.flatten()  # 2D 배열을 1D 배열로 변환

# 각 변수에 대해 히스토그램 그리기
for i, var in enumerate(continuous_vars):
    ax = axes[i]
    for label in data['passorfail'].unique():
        subset = data[data['passorfail'] == label]
        ax.hist(subset[var], bins=30, alpha=0.5, label=f'{label}')
    ax.set_title(f'Histogram of {var}')
    ax.set_xlabel(var)
    ax.set_ylabel('Frequency')
    ax.legend(title='passorfail')

# 빈 서브플롯 숨기기
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# 레이아웃 조정
plt.tight_layout()
plt.show()

# 'molten_temp'의 값별 개수 계산
molten_temp_counts = data['molten_temp'].value_counts()

# 결과 출력
print(molten_temp_counts)
import matplotlib.pyplot as plt

# 'molten_temp'의 값별 개수 계산
molten_temp_counts = data['molten_temp'].value_counts()

# 막대그래프 그리기
plt.figure(figsize=(15, 6))
molten_temp_counts.plot(kind='bar')
plt.title('Count of Each molten_temp Value')
plt.xlabel('molten_temp')
plt.ylabel('Count')
plt.show()

import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv("https://firebasestorage.googleapis.com/v0/b/ls-storage-e452a.appspot.com/o/casting.csv?alt=media&token=54fffc33-03ad-4431-8e20-34ede32c043f",
                   encoding='cp949',
                   index_col=0)

# 'molten_temp'의 값별 개수 계산
molten_temp_counts = data['molten_temp'].value_counts()

# 값이 5개 이하인 항목 필터링
rare_molten_temp = molten_temp_counts[molten_temp_counts <= 5].index

# 원본 데이터프레임에서 해당 항목들만 추출
filtered_data = data[data['molten_temp'].isin(rare_molten_temp)]

# 히스토그램 그리기
plt.figure(figsize=(12, 6))

for label in filtered_data['passorfail'].unique():
    subset = filtered_data[filtered_data['passorfail'] == label]
    plt.hist(subset['molten_temp'], bins=30, alpha=0.5, label=f'{label}')

plt.title('Histogram of molten_temp (5 or fewer) by passorfail')
plt.xlabel('molten_temp')
plt.ylabel('Frequency')
plt.legend(title='passorfail')
plt.show()
# 데이터 로드
data = pd.read_csv("https://firebasestorage.googleapis.com/v0/b/ls-storage-e452a.appspot.com/o/casting.csv?alt=media&token=54fffc33-03ad-4431-8e20-34ede32c043f",
                   encoding='cp949',
                   index_col=0)
# 'cast_pressure'의 값별 개수 계산
c_p_count = data[data['cast_pressure']<=249.5]['passorfail'].value_counts()

# 결과 출력
print(c_p_count)
# 'physical_strength'의 값별 개수 계산
p_s_count = data[data['physical_strength']<=656.5]['passorfail'].value_counts()

# 결과 출력
print(p_s_count)
# 'upper_mold_temp1'의 값별 개수 계산
umt_count = data[data['upper_mold_temp1']<=656.5]['passorfail'].value_counts()

# 결과 출력
print(umt_count)
# 'biscuit_thickness'의 값별 개수 계산
biscuit_count = data[data['biscuit_thickness']<=38.5]['passorfail'].value_counts()

# 결과 출력
print(biscuit_count)
# 'molten_temp'의 값별 개수 계산
biscuit100_count = data[data['biscuit_thickness']>=61]['biscuit_thickness'].value_counts()

# 결과 출력
print(biscuit100_count)

### mode_code 별로 데이터 분리
import pandas as pd

# 데이터 로드

data_url = "https://firebasestorage.googleapis.com/v0/b/ls-storage-e452a.appspot.com/o/casting.csv?alt=media&token=54fffc33-03ad-4431-8e20-34ede32c043f"
data = pd.read_csv(data_url, encoding='cp949', index_col=0)


data['mold_code'].value_counts()

mold_8917 = data[data['mold_code'] == 8917].copy().reset_index(drop=True)
mold_8722 = data[data['mold_code'] == 8722].copy().reset_index(drop=True)
mold_8600 = data[data['mold_code'] == 8600].copy().reset_index(drop=True)
mold_8576 = data[data['mold_code'] == 8576].copy().reset_index(drop=True)
mold_8573 = data[data['mold_code'] == 8573].copy().reset_index(drop=True)
mold_8413 = data[data['mold_code'] == 8413].copy().reset_index(drop=True)
mold_8412 = data[data['mold_code'] == 8412].copy().reset_index(drop=True)
### trial열 추가
# 'trial' 열 추가
mold_8412['trial'] = 0

# 인덱스를 차례대로 하나씩 넘기면서 'count' 변수를 비교
trial_counter = 0
previous_count = None

for index, row in mold_8412.iterrows():
    if previous_count is not None and abs(row['count'] - previous_count) >= 5:
        trial_counter += 1
    mold_8412.at[index, 'trial'] = trial_counter
    previous_count = row['count']

# 결과 출력
mold_8412.head(25)
cols = list(mold_8412.columns)
cols.insert(cols.index('trial'), cols.pop(cols.index('count')))
mold_8412 = mold_8412[cols]

mold_8412.head(200)
from google.colab import drive
import pandas as pd

# Google 드라이브 마운트
drive.mount('/content/drive')

# 데이터프레임을 CSV 파일로 저장
file_path = '/content/drive/MyDrive/lsda02/data/mold_8412_with_trials.csv'
mold_8412.to_csv(file_path, encoding='utf-8-sig', index=False)

print(f'File saved to {file_path}')

### count 히스토그램 그리기
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드 (로컬 파일 경로 사용)
data_url = "https://firebasestorage.googleapis.com/v0/b/ls-storage-e452a.appspot.com/o/casting.csv?alt=media&token=54fffc33-03ad-4431-8e20-34ede32c043f"
data = pd.read_csv(data_url, encoding='cp949', index_col=0)

# 'mold_code'가 8722인 데이터 필터링
mold_8722 = data[data['mold_code'] == 8722]

# 'passorfail' 값을 기준으로 'count' 히스토그램 그리기
plt.figure(figsize=(12, 6))

for label in mold_8722['passorfail'].unique():
    subset = mold_8722[mold_8722['passorfail'] == label]
    plt.hist(subset['count'], bins=30, alpha=0.5, label=f'{label}')

plt.title('Histogram of count for mold_code 8722 by passorfail')
plt.xlabel('count')
plt.ylabel('Frequency')
plt.legend(title='passorfail')
plt.show()

use_col = data.dtypes[data.dtypes!='object'].index
data = data[use_col]
data
data.info()
plt.figure(figsize=(40, 40))
sns.heatmap(data.corr(), annot = True, fmt = '.1f', square = True, cmap = 'Blues')
plt.show()
use_col = data.dtypes[data.dtypes!='object'].index
data = data[use_col]
data

## count 초반후반 불량발생
### 8722
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8722_with_trials.csv')

# 'trial'이 3, 4, 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([3, 4, 5, 6])]

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure'])

# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)
axes = axes.flatten()

trial_values = [3, 4, 5, 6]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8722')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()

plt.tight_layout()
plt.show()


import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8722_with_trials.csv')

# 'trial'이 3, 4, 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([7, 8, 9, 10])]

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure'])

# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)
axes = axes.flatten()

trial_values = [7, 8, 9, 10]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8722')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()

plt.tight_layout()
plt.show()


import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8722_with_trials.csv')

# 'trial'이 3, 4, 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([11, 12, 13, 14])]

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure'])

# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)
axes = axes.flatten()

trial_values = [11, 12, 13, 14]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8722')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()

plt.tight_layout()
plt.show()


### 8917
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8917_with_trials.csv')

# 'trial'이 3, 4, 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([3, 4, 5, 6])]

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure'])

# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)
axes = axes.flatten()

trial_values = [3, 4, 5, 6]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()

plt.tight_layout()
plt.show()


### 8600
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8600_with_trials.csv')

# 'trial'이 3, 4, 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([3, 4, 5, 6])]

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure'])

# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)
axes = axes.flatten()

trial_values = [3, 4, 5, 6]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8600')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()

plt.tight_layout()
plt.show()


### 8576
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8576_with_trials.csv')

# 'trial'이 3, 4, 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([1,2,3,4])]

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure'])

# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)
axes = axes.flatten()

trial_values = [1,2,3,4]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8576')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()

plt.tight_layout()
plt.show()


### 8573
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8573_with_trials.csv')

# 'trial'이 3, 4, 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([3, 4, 5, 6])]

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure'])

# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)
axes = axes.flatten()

trial_values = [3, 4, 5, 6]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8573')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()

plt.tight_layout()
plt.show()


### 8413
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8413_with_trials.csv')

# 'trial'이 3, 4, 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([3, 4, 5, 6])]

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure'])

# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)
axes = axes.flatten()

trial_values = [3, 4, 5, 6]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8413')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()

plt.tight_layout()
plt.show()


### 8412
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8412_with_trials.csv')

# 'trial'이 3, 4, 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([3, 4, 5, 6])]

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure'])

# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)
axes = axes.flatten()

trial_values = [3, 4, 5, 6]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8412')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()

plt.tight_layout()
plt.show()


## cast_pressure가 정상범위인데 불량 발생하는 이유는 뭘까
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8917_with_trials.csv')
vertical_lines = [50, 100, 150, 200, 250]  # 세로선을 그릴 위치

# 'trial'이 3, 4, 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([ 5, 6])]


# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)
axes = axes.flatten()

trial_values = [ 5, 6]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure','high_section_speed'])

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 그리기
    for line in vertical_lines:
        ax.axvline(x=line, color='gray', linestyle='--', linewidth=1)

for i, trial in enumerate(trial_values):
    ax = axes[i+2]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['high_section_speed'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('high_section_speed')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 그리기
    for line in vertical_lines:
        ax.axvline(x=line, color='gray', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()


import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8917_with_trials.csv')
vertical_lines = [50, 100, 150, 200, 250]  # 세로선을 그릴 위치

# 'trial'이 3, 4, 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([ 5, 6])]


# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)
axes = axes.flatten()

trial_values = [ 5, 6]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure','biscuit_thickness'])

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 그리기
    for line in vertical_lines:
        ax.axvline(x=line, color='gray', linestyle='--', linewidth=1)

for i, trial in enumerate(trial_values):
    ax = axes[i+2]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['biscuit_thickness'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('biscuit_thickness')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 그리기
    for line in vertical_lines:
        ax.axvline(x=line, color='gray', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()


import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8917_with_trials.csv')
vertical_lines = [50, 100, 150, 200, 250]  # 세로선을 그릴 위치

# 'trial'이 3, 4, 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([ 5, 6])]


# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)
axes = axes.flatten()

trial_values = [ 5, 6]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure','upper_mold_temp1'])

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 그리기
    for line in vertical_lines:
        ax.axvline(x=line, color='gray', linestyle='--', linewidth=1)

for i, trial in enumerate(trial_values):
    ax = axes[i+2]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['upper_mold_temp1'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('upper_mold_temp1')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 그리기
    for line in vertical_lines:
        ax.axvline(x=line, color='gray', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()


import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8917_with_trials.csv')
vertical_lines = [50, 100, 150, 200, 250]  # 세로선을 그릴 위치

# 'trial'이 3, 4, 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([ 5, 6])]


# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)
axes = axes.flatten()

trial_values = [ 5, 6]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure','upper_mold_temp2'])

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 그리기
    for line in vertical_lines:
        ax.axvline(x=line, color='gray', linestyle='--', linewidth=1)

for i, trial in enumerate(trial_values):
    ax = axes[i+2]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['upper_mold_temp2'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('upper_mold_temp2')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 그리기
    for line in vertical_lines:
        ax.axvline(x=line, color='gray', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()


import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8917_with_trials.csv')
vertical_lines = [50, 100, 150, 200, 250]  # 세로선을 그릴 위치

# 'trial'이 3, 4, 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([ 5, 6])]


# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True)
axes = axes.flatten()

trial_values = [ 5, 6]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure','upper_mold_temp3'])

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 그리기
    for line in vertical_lines:
        ax.axvline(x=line, color='gray', linestyle='--', linewidth=1)

for i, trial in enumerate(trial_values):
    ax = axes[i+2]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['upper_mold_temp3'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('upper_mold_temp3')
    ax.legend()
    ax.set_ylim(1100, 1500)  # y축 범위 고정
    # 세로선 그리기
    for line in vertical_lines:
        ax.axvline(x=line, color='gray', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()


import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8917_with_trials.csv')
vertical_lines = [50, 100, 150, 200, 250]  # 세로선을 그릴 위치

# 'trial'이 3, 4, 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([ 5, 6])]


# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)
axes = axes.flatten()

trial_values = [ 5, 6]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure','lower_mold_temp1'])

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 그리기
    for line in vertical_lines:
        ax.axvline(x=line, color='gray', linestyle='--', linewidth=1)

for i, trial in enumerate(trial_values):
    ax = axes[i+2]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['lower_mold_temp1'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('lower_mold_temp1')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 그리기
    for line in vertical_lines:
        ax.axvline(x=line, color='gray', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()


import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8917_with_trials.csv')
vertical_lines = [50, 100, 150, 200, 250]  # 세로선을 그릴 위치

# 'trial'이 3, 4, 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([ 5, 6])]


# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)
axes = axes.flatten()

trial_values = [ 5, 6]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure','lower_mold_temp2'])

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 그리기
    for line in vertical_lines:
        ax.axvline(x=line, color='gray', linestyle='--', linewidth=1)

for i, trial in enumerate(trial_values):
    ax = axes[i+2]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['lower_mold_temp2'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('lower_mold_temp2')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 그리기
    for line in vertical_lines:
        ax.axvline(x=line, color='gray', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()


import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8917_with_trials.csv')
vertical_lines = [50, 100, 150, 200, 250]  # 세로선을 그릴 위치

# 'trial'이 3, 4, 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([ 5, 6])]


# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True)
axes = axes.flatten()

trial_values = [ 5, 6]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure','lower_mold_temp3'])

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 그리기
    for line in vertical_lines:
        ax.axvline(x=line, color='gray', linestyle='--', linewidth=1)

for i, trial in enumerate(trial_values):
    ax = axes[i+2]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['lower_mold_temp3'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('lower_mold_temp3')
    ax.legend()
    ax.set_ylim(1100, 1500)  # y축 범위 고정
    # 세로선 그리기
    for line in vertical_lines:
        ax.axvline(x=line, color='gray', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()


import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8917_with_trials.csv')
vertical_lines = [50, 100, 150, 200, 250]  # 세로선을 그릴 위치

# 'trial'이 3, 4, 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([ 5, 6])]


# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True)
axes = axes.flatten()

trial_values = [ 5, 6]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure','sleeve_temperature'])

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 그리기
    for line in vertical_lines:
        ax.axvline(x=line, color='gray', linestyle='--', linewidth=1)

for i, trial in enumerate(trial_values):
    ax = axes[i+2]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['sleeve_temperature'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('sleeve_temperature')
    ax.legend()
    ax.set_ylim(100, 500)  # y축 범위 고정
    # 세로선 그리기
    for line in vertical_lines:
        ax.axvline(x=line, color='gray', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()


import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8917_with_trials.csv')
vertical_lines = [50, 100, 150, 200, 250]  # 세로선을 그릴 위치

# 'trial'이 3, 4, 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([ 5, 6])]


# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True)
axes = axes.flatten()

trial_values = [ 5, 6]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure','physical_strength'])

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 그리기
    for line in vertical_lines:
        ax.axvline(x=line, color='gray', linestyle='--', linewidth=1)

for i, trial in enumerate(trial_values):
    ax = axes[i+2]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['physical_strength'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=0.6)
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('physical_strength')
    ax.legend()
    ax.set_ylim(400, 800)  # y축 범위 고정
    # 세로선 그리기
    for line in vertical_lines:
        ax.axvline(x=line, color='gray', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()


## gap(시간간격을 기록하는)열을 생성
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8412_with_trials.csv')
data.info()
# 'date' 열의 데이터 타입을 시간으로 변경 (시:분:초 형식)
data['date'] = pd.to_datetime(data['date'], format='%H:%M:%S')
# 'gap'이라는 열을 추가 (기본값으로 0을 추가)
data['gap'] = 0
data.info()
# 'date' 열의 데이터 타입을 시간으로 변경 (시:분:초 형식)
# '1900-01-01 08:01:39' 형식에서 시간만 추출하여 새로운 열 'time'으로 저장
data['time'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S').dt.time

# 'gap'이라는 열을 추가하여 앞 행과의 시간 차이를 기록 (초 단위)
data['gap'] = pd.to_datetime(data['time'], format='%H:%M:%S').diff().dt.total_seconds().fillna(0)

# 음수 값이 나오는 경우 86,400초(1일)을 더하여 수정
data['gap'] = data['gap'].apply(lambda x: x if x >= 0 else x + 86400)

data.head()
# 데이터프레임을 CSV 파일로 저장
file_path = '/content/drive/MyDrive/lsda02/data/mold_8412_with_trials_gap.csv'
data.to_csv(file_path, encoding='utf-8-sig', index=False)

print(f'File saved to {file_path}')

## gap_sign 열을 추가
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8412_with_trials_gap.csv')
data['gap_sign'] = data['gap'].apply(lambda x: 1 if x >= 170 else 0)
# 데이터프레임을 CSV 파일로 저장
file_path = '/content/drive/MyDrive/lsda02/data/mold_8412_with_trials_gap_gapsign.csv'
data.to_csv(file_path, encoding='utf-8-sig', index=False)

print(f'File saved to {file_path}')

## gap_sign=1일때 세로선 추가
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8917_with_trials_gap_gapsign.csv')

# 'trial'이 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([5, 6])]

# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)
axes = axes.flatten()

trial_values = [5, 6]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure', 'lower_mold_temp1'])

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=alphas[int(passorfail)])
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 추가
    for idx, row in subset.iterrows():
        if row['gap_sign'] == 1:
            ax.axvline(x=row['count'], color='gray', linestyle='--', linewidth=1)

for i, trial in enumerate(trial_values):
    ax = axes[i + 2]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['lower_mold_temp1'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=alphas[int(passorfail)])
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('Lower Mold Temp 1')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 추가
    for idx, row in subset.iterrows():
        if row['gap_sign'] == 1:
            ax.axvline(x=row['count'], color='gray', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()

import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8917_with_trials_gap_gapsign.csv')

# 'trial'이 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([5, 6])]

# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)
axes = axes.flatten()

trial_values = [5, 6]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure', 'lower_mold_temp2'])

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=alphas[int(passorfail)])
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 추가
    for idx, row in subset.iterrows():
        if row['gap_sign'] == 1:
            ax.axvline(x=row['count'], color='gray', linestyle='--', linewidth=1)

for i, trial in enumerate(trial_values):
    ax = axes[i + 2]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['lower_mold_temp2'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=alphas[int(passorfail)])
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('Lower Mold Temp 2')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 추가
    for idx, row in subset.iterrows():
        if row['gap_sign'] == 1:
            ax.axvline(x=row['count'], color='gray', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()

import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8917_with_trials_gap_gapsign.csv')

# 'trial'이 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([5, 6])]

# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)
axes = axes.flatten()

trial_values = [5, 6]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure', 'upper_mold_temp2'])

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=alphas[int(passorfail)])
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 추가
    for idx, row in subset.iterrows():
        if row['gap_sign'] == 1:
            ax.axvline(x=row['count'], color='gray', linestyle='--', linewidth=1)

for i, trial in enumerate(trial_values):
    ax = axes[i + 2]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['upper_mold_temp2'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=alphas[int(passorfail)])
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('Upper Mold Temp 2')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 추가
    for idx, row in subset.iterrows():
        if row['gap_sign'] == 1:
            ax.axvline(x=row['count'], color='gray', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()

import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('/content/drive/MyDrive/lsda02/data/mold_8917_with_trials_gap_gapsign.csv')

# 'trial'이 5, 6인 데이터 필터링
filtered_data = data[data['trial'].isin([5, 6])]

# 'trial'별로 산점도 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True, sharey=True)
axes = axes.flatten()

trial_values = [5, 6]
colors = {0: 'skyblue', 1: 'orange'}
alphas = {0: 0.1, 1: 1.0}  # 투명도 설정

# 결측값 제거
filtered_data = filtered_data.dropna(subset=['passorfail', 'count', 'cast_pressure', 'upper_mold_temp1'])

for i, trial in enumerate(trial_values):
    ax = axes[i]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['cast_pressure'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=alphas[int(passorfail)])
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('Cast Pressure')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 추가
    for idx, row in subset.iterrows():
        if row['gap_sign'] == 1:
            ax.axvline(x=row['count'], color='gray', linestyle='--', linewidth=1)

for i, trial in enumerate(trial_values):
    ax = axes[i + 2]
    subset = filtered_data[filtered_data['trial'] == trial]
    for passorfail in subset['passorfail'].unique():
        trial_subset = subset[subset['passorfail'] == passorfail]
        ax.scatter(trial_subset['count'], trial_subset['upper_mold_temp1'],
                   c=colors[int(passorfail)], label=f'passorfail {int(passorfail)}', alpha=alphas[int(passorfail)])
    ax.set_title(f'Trial {trial} of mold_8917')
    ax.set_xlabel('Count')
    ax.set_ylabel('Upper Mold Temp 1')
    ax.legend()
    ax.set_ylim(0, 400)  # y축 범위 고정
    # 세로선 추가
    for idx, row in subset.iterrows():
        if row['gap_sign'] == 1:
            ax.axvline(x=row['count'], color='gray', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()



# 12번째 시도 - 'EMS_operation_time' 제거 X, 이상치 제거 X, 제일 덜 중요한 변수 2개 제거 + mold_code 변수 제거 안 한 버전


## 변수 제거 및 결측치, 이상치 처리

In [ ]:
## 변수 제거 및 결측치, 이상치 처리
pip install eli5
# 패키지 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn import datasets, svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn import set_config
from sklearn.utils import resample
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_validate
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc
from IPython.display import Image
from numpy import interp
import scipy.stats


import scipy
import xgboost
import lightgbm
import warnings
from eli5.sklearn import PermutationImportance
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from lightgbm import LGBMClassifier, plot_importance
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedShuffleSplit

from IPython.display import Image
from sklearn.cluster import KMeans
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings("ignore")

!apt-get -qq install fonts-nanum

import os
import matplotlib.pyplot as plt
from matplotlib import font_manager
import matplotlib.font_manager as fm

fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')                        # 원하는 폰트 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가

plt.rcParams.update({'font.size': 9, 'font.family': 'NanumGothic'}) # 폰트 설정
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("https://firebasestorage.googleapis.com/v0/b/ls-storage-e452a.appspot.com/o/casting.csv?alt=media&token=54fffc33-03ad-4431-8e20-34ede32c043f",
                      encoding='cp949',
                      index_col=0)

data = df.copy()
data.info()
# 정보 손실된 행 1개 확인 및 제거 (거의 대부분의 열에서 공통으로 결측치 발생한 열)

missing_passorfail = data[data['passorfail'].isna()]
missing_passorfail
data = data.dropna(subset=['passorfail'])
data.info()
data.isnull().sum()
# 결측치가 50% 이상인 열들 제거
columns_to_drop = ['molten_volume', 'tryshot_signal', 'heating_furnace']
data = data.drop(columns = columns_to_drop)

data
# working 이라는 컬럼에 '정지' 라는 값이 51개 존재
# 그러나 '정지'값은 전체의 0.055%임 - 매우매우매우 작은 값이라 무시해도 된다 - 따라서 working 버려!!!
## (51/92014 = 0.0005542634816440976)

data['working'].value_counts(dropna = False)
# working 컬럼 삭제 (위 설명 참고)
columns_to_drop1 = ['working']
data = data.drop(columns = columns_to_drop1)

data
data.info()
### date_time 변환 안 하고 시계열 변수 라벨인코딩함 - 이번에는 date, time 모두 사용
# data['date'] = pd.to_datetime(data['date'])
# data['registration_time'] = pd.to_datetime(data['registration_time'])
# data['time'] = pd.to_datetime(data['time'])

# columns_to_drop1 = ['time']
# data = data.drop(columns = columns_to_drop1)
# data = data.select_dtypes(exclude=['object'])
# LabelEncoder 클래스를 불러옵니다.
from sklearn.preprocessing import LabelEncoder

# object 타입의 컬럼 리스트로 가져오기
object_columns = data.select_dtypes(include=['object']).columns.tolist()

# 각 object 타입의 컬럼에 대해 라벨 인코딩 수행
label_encoders = {}
for column in object_columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

data
plt.figure(figsize=(25, 25))
sns.heatmap(data.corr(), annot = True, fmt = '.1f', square = True, cmap = 'Blues')
plt.show()
# 히트맵을 통해 'line', 'name', 'mold_name', 'emergency_stop'은 하나의 값만 가지는 걸 알 수 있음
# 추가로, 'time' 과 'registration_time' 상관계수가 1, 즉 아예 똑같은 값을 가지고 있다는 걸 알 수 있다.
# 이 5개의 컬럼 전부 삭제



# working 컬럼 삭제 (위 설명 참고)
columns_to_drop1 = ['line', 'name', 'mold_name', 'emergency_stop','registration_time']
data = data.drop(columns = columns_to_drop1)
data.info()
# plt.figure(figsize=(25, 25))
# sns.heatmap(data.corr(), annot = True, fmt = '.1f', square = True, cmap = 'Blues')
# plt.show()
data.info()
data['molten_temp'].value_counts(dropna = False)
# # 'molten_time' 값이 100 미만인 값의 개수
# count_below_100 = (df['molten_temp'] < 100).sum()

# # 'molten_time' 값이 100 이상인 값의 개수
# count_100_and_above = (df['molten_temp'] >= 100).sum()

# print("100 미만인 값의 개수:", count_below_100)
# print("100 이상인 값의 개수:", count_100_and_above)
# # 산점도 그리기
# plt.figure(figsize=(10, 6))
# plt.scatter(df['time'], df['molten_temp'], color='blue')
# plt.title('Molten Temperature Over Time')
# plt.xlabel('Time')
# plt.ylabel('Molten Temperature')
# plt.grid(True)
# plt.show()


# # passorfail 이 0이면 빨강색, 1이면 초록색으로 표시하는 코드 추가해줘
# # 'molten_temp'가 100 미만인 인덱스 찾기
# indices_below_100 = df.index[df['molten_temp'] < 100].tolist()

# # 시각화
# plt.figure(figsize=(12, 8))

# for idx in indices_below_100:
#     start_idx = max(0, idx - 10)
#     end_idx = min(len(df), idx + 11)
#     subset = df.iloc[start_idx:end_idx]

#     plt.plot(subset['time'], subset['molten_temp'], marker='o', label=f'Index {idx}')

# plt.title('Molten Temperature Over Time (100 미만인 값 전후 10개의 데이터)')
# plt.xlabel('Time')
# plt.ylabel('Molten Temperature')
# plt.legend()
# plt.grid(True)
# plt.show()
data.dropna(inplace=True)

data.info()
# # 데이터 타입이 object인 열만 제거
# data = data.select_dtypes(exclude=['object'])

# data_copy = data.copy()
# data_copy.reset_index(drop=True, inplace=True)

# data_copy
# 수치형 변수들만 남기고 나머지 제거

num_columns = data.select_dtypes(include=np.number).columns.tolist()
data = data[num_columns]

data_copy = data.copy()
data_copy.reset_index(drop=True, inplace=True)

data_copy
### 여기 다르게 한 부분임
# # 이상치 제거 (0.1% 미만, 99.9% 초과인 값을 제거함)

# for col in data_copy.dtypes[data_copy.dtypes!='object'].index:
#     UCL = np.percentile(data_copy[col], 99.9)
#     LCL = np.percentile(data_copy[col], 0.1)
#     data_copy = data_copy[(data_copy[col] <= UCL) & (data_copy[col] >= LCL)]

# data_copy.reset_index(drop=True, inplace=True)
# data_copy
# 전자 교반 : 융해(融解) 금속에 강한 자기장을 가하고 동시에 도가니 양 끝에 있는 전극에 직류 전기를 보내어 합금하는 방법
# 'EMS_operation_time' = 전자교반 작동시간
# 값이 0, 3, 6, 23 밖에 안 나옴 - 의미 없어 보임

data_copy['EMS_operation_time'].value_counts(dropna = False)
data_copy[data_copy['EMS_operation_time'] == 23]['passorfail'].value_counts()
### 여기 다르게 한 부분임 2 - mold_code 를 제거하지 않음!!!!
# data_copy.drop(['count', 'EMS_operation_time', 'mold_code'], axis=1, inplace=True)
# data_copy

data_copy.drop(['count'], axis=1, inplace=True)
data_copy
### 여기 다르게 한 부분임 3
# 가장 영향이 없는 변수 2개를 미리 제거해본 경우

data_copy.drop(['upper_mold_temp3', 'lower_mold_temp3'], axis=1, inplace=True)
data_copy

## 모델링 시작

In [ ]:
## 모델링 시작
X = data_copy.drop('passorfail', axis=1).values
y = data_copy['passorfail'].values
sss = StratifiedShuffleSplit(n_splits=5, random_state=42, test_size=0.2)

mms = MinMaxScaler()
dt_clf = DecisionTreeClassifier(random_state=42)
rf_clf = RandomForestClassifier(random_state=42)
xgb_clf = XGBClassifier(random_state=42, eval_metric='error', use_label_encoder=False)
lgbm_clf = LGBMClassifier(random_state=42)
ada_clf = AdaBoostClassifier(random_state=42)
dt_clf = DecisionTreeClassifier(random_state=42)
dt_fold_f1 = []
n_iter = 0
print('모델 :', dt_clf)
idx_list = []

for train_idx, test_idx in sss.split(X, y):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    for train_idx, valid_idx in sss.split(X_train, y_train):
        X_valid = X_train[valid_idx]
        y_valid = y_train[valid_idx]

    X_train_scaled = mms.fit_transform(X_train)
    X_test_scaled = mms.transform(X_test)
    X_valid_scaled = mms.transform(X_valid)

    dt_clf.fit(X_train_scaled, y_train)
    valid_pred = dt_clf.predict(X_valid_scaled)
    test_pred = dt_clf.predict(X_test_scaled)

    valid_f1 = f1_score(y_valid, valid_pred)
    test_f1 = f1_score(y_test, test_pred)

    n_iter += 1
    dt_fold_f1.append([n_iter, test_f1])
    idx_list.append([train_idx, test_idx])

    print(f'StratifiedShuffleSplit {n_iter} 검증 데이터 f1 정확도 : {valid_f1}')
    print(f'StratifiedShuffleSplit {n_iter} 테스트 데이터 f1 정확도 : {test_f1}')
    print('='*50)

dt_fold_f1.sort(key=lambda x : x[1], reverse=True)
print(f'교차검증 중 가장 높은 정확도 : Split {dt_fold_f1[0][0]} {dt_fold_f1[0][1]}')
dt_clf.fit(X[idx_list[dt_fold_f1[0][0]-1][0]], y[idx_list[dt_fold_f1[0][0]-1][0]])
pred = dt_clf.predict(X[idx_list[dt_fold_f1[0][0]-1][1]])
cm = confusion_matrix(y[idx_list[dt_fold_f1[0][0]-1][1]], pred)
sns.heatmap(cm, annot=True, fmt='.0f')
plt.show()
rf_clf = RandomForestClassifier(random_state=42)
rf_fold_f1 = []
n_iter = 0
print('모델 :', rf_clf)
idx_list = []

for train_idx, test_idx in sss.split(X, y):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    for train_idx, valid_idx in sss.split(X_train, y_train):
        X_valid = X_train[valid_idx]
        y_valid = y_train[valid_idx]

    X_train_scaled = mms.fit_transform(X_train)
    X_test_scaled = mms.transform(X_test)
    X_valid_scaled = mms.transform(X_valid)

    rf_clf.fit(X_train_scaled, y_train)
    valid_pred = rf_clf.predict(X_valid_scaled)
    test_pred = rf_clf.predict(X_test_scaled)

    valid_f1 = f1_score(y_valid, valid_pred)
    test_f1 = f1_score(y_test, test_pred)

    n_iter += 1
    rf_fold_f1.append([n_iter, test_f1])
    idx_list.append([train_idx, test_idx])

    print(f'StratifiedShuffleSplit {n_iter} 검증 데이터 f1 정확도 : {valid_f1}')
    print(f'StratifiedShuffleSplit {n_iter} 테스트 데이터 f1 정확도 : {test_f1}')
    print('='*50)

rf_fold_f1.sort(key=lambda x : x[1], reverse=True)
print(f'교차검증 중 가장 높은 정확도 : Split {rf_fold_f1[0][0]} {rf_fold_f1[0][1]}')
rf_clf.fit(X[idx_list[rf_fold_f1[0][0]-1][0]], y[idx_list[rf_fold_f1[0][0]-1][0]])
pred = rf_clf.predict(X[idx_list[rf_fold_f1[0][0]-1][1]])
cm = confusion_matrix(y[idx_list[rf_fold_f1[0][0]-1][1]], pred)
sns.heatmap(cm, annot=True, fmt='.0f')
plt.show()
xgb_clf = XGBClassifier(random_state=42)
xgb_fold_f1 = []
n_iter = 0
print('모델 :', xgb_clf)
idx_list = []
column_names = data_copy.drop('passorfail', axis=1).columns.tolist()


for train_idx, test_idx in sss.split(X, y):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    for train_idx, valid_idx in sss.split(X_train, y_train):
        X_valid = X_train[valid_idx]
        y_valid = y_train[valid_idx]

    X_train_scaled = mms.fit_transform(X_train)
    X_test_scaled = mms.transform(X_test)
    X_valid_scaled = mms.transform(X_valid)

    xgb_clf.fit(X_train_scaled, y_train)
    valid_pred = xgb_clf.predict(X_valid_scaled)
    test_pred = xgb_clf.predict(X_test_scaled)

    valid_f1 = f1_score(y_valid, valid_pred)
    test_f1 = f1_score(y_test, test_pred)

    n_iter += 1
    xgb_fold_f1.append([n_iter, test_f1])
    idx_list.append([train_idx, test_idx])

    print(f'StratifiedShuffleSplit {n_iter} 검증 데이터 f1 정확도 : {valid_f1}')
    print(f'StratifiedShuffleSplit {n_iter} 테스트 데이터 f1 정확도 : {test_f1}')
    print('='*50)

xgb_fold_f1.sort(key=lambda x : x[1], reverse=True)
print(f'교차검증 중 가장 높은 정확도 : Split {xgb_fold_f1[0][0]} {xgb_fold_f1[0][1]}')
xgb_clf.fit(X[idx_list[xgb_fold_f1[0][0]-1][0]], y[idx_list[xgb_fold_f1[0][0]-1][0]])
pred = xgb_clf.predict(X[idx_list[xgb_fold_f1[0][0]-1][1]])
cm = confusion_matrix(y[idx_list[xgb_fold_f1[0][0]-1][1]], pred)
sns.heatmap(cm, annot=True, fmt='.0f')
plt.show()
X_test_scaled
import shap
import xgboost as xgb

# SHAP 설명자 생성
explainer = shap.Explainer(xgb_clf)
shap_values = explainer.shap_values(X_train_scaled)

# 특성별 SHAP 요약 플롯 생성
shap.summary_plot(shap_values, X_train_scaled, feature_names=column_names, color_bar=False)

lgbm_clf_clf = LGBMClassifier(random_state=42)
lgbm_fold_f1 = []
n_iter = 0
print('모델 :', lgbm_clf)
idx_list = []

for train_idx, test_idx in sss.split(X, y):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    for train_idx, valid_idx in sss.split(X_train, y_train):
        X_valid = X_train[valid_idx]
        y_valid = y_train[valid_idx]

    X_train_scaled = mms.fit_transform(X_train)
    X_test_scaled = mms.transform(X_test)
    X_valid_scaled = mms.transform(X_valid)

    lgbm_clf.fit(X_train_scaled, y_train)
    valid_pred = lgbm_clf.predict(X_valid_scaled)
    test_pred = lgbm_clf.predict(X_test_scaled)

    valid_f1 = f1_score(y_valid, valid_pred)
    test_f1 = f1_score(y_test, test_pred)

    n_iter += 1
    lgbm_fold_f1.append([n_iter, test_f1])
    idx_list.append([train_idx, test_idx])

    print(f'StratifiedShuffleSplit {n_iter} 검증 데이터 f1 정확도 : {valid_f1}')
    print(f'StratifiedShuffleSplit {n_iter} 테스트 데이터 f1 정확도 : {test_f1}')
    print('='*50)

lgbm_fold_f1.sort(key=lambda x : x[1], reverse=True)
print(f'교차검증 중 가장 높은 정확도 : Split {lgbm_fold_f1[0][0]} {lgbm_fold_f1[0][1]}')
lgbm_clf.fit(X[idx_list[lgbm_fold_f1[0][0]-1][0]], y[idx_list[lgbm_fold_f1[0][0]-1][0]])
pred = lgbm_clf.predict(X[idx_list[lgbm_fold_f1[0][0]-1][1]])
cm = confusion_matrix(y[idx_list[lgbm_fold_f1[0][0]-1][1]], pred)
sns.heatmap(cm, annot=True, fmt='.0f')
plt.show()
ada_clf = AdaBoostClassifier(random_state=42)
ada_fold_f1 = []
n_iter = 0
print('모델 :', ada_clf)
idx_list = []

for train_idx, test_idx in sss.split(X, y):
  X_train, X_test = X[train_idx], X[test_idx]
  y_train, y_test = y[train_idx], y[test_idx]

  for train_idx, valid_idx in sss.split(X_train, y_train):
    X_valid = X_train[valid_idx]
    y_valid = y_train[valid_idx]

  X_train_scaled = mms.fit_transform(X_train)
  X_test_scaled = mms.transform(X_test)
  X_valid_scaled = mms.transform(X_valid)

  ada_clf.fit(X_train_scaled, y_train)
  valid_pred = ada_clf.predict(X_valid_scaled)
  test_pred = ada_clf.predict(X_test_scaled)

  valid_f1 = f1_score(y_valid, valid_pred)
  test_f1 = f1_score(y_test, test_pred)

  n_iter += 1
  ada_fold_f1.append([n_iter, test_f1])
  idx_list.append([train_idx, test_idx])

  print(f'StratifiedShuffleSplit {n_iter} 검증 데이터 f1 정확도 : {valid_f1}')
  print(f'StratifiedShuffleSplit {n_iter} 테스트 데이터 f1 정확도 : {test_f1}')
  print('='*50)

ada_fold_f1.sort(key=lambda x : x[1], reverse=True)
print(f'교차검증 중 가장 높은 정확도 : Split {ada_fold_f1[0][0]} {ada_fold_f1[0][1]}')
ada_clf.fit(X[idx_list[ada_fold_f1[0][0]-1][0]], y[idx_list[ada_fold_f1[0][0]-1][0]])
pred = ada_clf.predict(X[idx_list[ada_fold_f1[0][0]-1][1]])
cm = confusion_matrix(y[idx_list[ada_fold_f1[0][0]-1][1]], pred)
sns.heatmap(cm, annot=True, fmt='.0f')
plt.show()
pi = PermutationImportance(xgb_clf, scoring='f1', n_iter=30)
pi.fit(X[idx_list[xgb_fold_f1[0][0]-1][0]], y[idx_list[xgb_fold_f1[0][0]-1][0]])
permu_df = pd.DataFrame()
permu_df["feature"] = data_copy.columns[:-1]
permu_df["importance_weight"] = pi.feature_importances_
permu_df.sort_values(by='importance_weight', ascending=False, inplace=True)
permu_df.reset_index(drop=True, inplace=True)
plt.figure(figsize=(10, 8))
sns.barplot(x='importance_weight', y='feature', data=permu_df)
plt.title('Permutaion Importance', fontsize=18)
plt.show()
from xgboost import XGBClassifier, plot_tree
xgb = XGBClassifier(random_state=42)
tmp = pd.DataFrame(X[idx_list[xgb_fold_f1[0][0]-1][0]], columns=data_copy.columns[:-1])
xgb.fit(tmp, y[idx_list[xgb_fold_f1[0][0]-1][0]])
# 트리 시각화
plt.figure(figsize=(20, 15))
plot_tree(xgb, num_trees=0, rankdir='TB')
plt.show()
# XGB : Split 5 0.922386144964721